# Lab 9: CellRank Fate Probabilities

**Module 9** - Predicting Cell Fates with CellRank

## Objectives
- Compute transition probabilities
- Identify terminal states
- Calculate fate probabilities
- Find driver genes


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

try:
    import cellrank as cr
    import scvelo as scv
    CELLRANK_AVAILABLE = True
except ImportError:
    CELLRANK_AVAILABLE = False
    print("CellRank not installed. Run: pip install cellrank")

if CELLRANK_AVAILABLE:
    scv.settings.set_figure_params(dpi=100, facecolor='white')
    
    # Load data with velocity
    adata = scv.datasets.pancreas()
    scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
    scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
    scv.tl.velocity(adata)
    scv.tl.velocity_graph(adata)
    
    print("Data prepared for CellRank")


In [ ]:
if CELLRANK_AVAILABLE:
    # Create CellRank kernel from velocity
    vk = cr.kernels.VelocityKernel(adata)
    vk.compute_transition_matrix()
    
    print("Velocity kernel computed")
    print(f"Transition matrix shape: {vk.transition_matrix.shape}")


In [ ]:
if CELLRANK_AVAILABLE:
    # Identify terminal states using GPCCA
    g = cr.estimators.GPCCA(vk)
    g.compute_schur(n_components=10)
    g.compute_macrostates(n_states=4, cluster_key='clusters')
    
    # Plot macrostates
    g.plot_macrostates()


In [ ]:
if CELLRANK_AVAILABLE:
    # Set terminal states and compute fate probabilities
    g.set_terminal_states_from_macrostates()
    g.compute_fate_probabilities()
    
    # Plot fate probabilities
    g.plot_fate_probabilities()
